<a href="https://colab.research.google.com/github/JacksonLeb/tensorflow_learning/blob/main/08_rnn_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#introduction to NLP fundamentals in Tf

##Getting info from natural language (sequences or speech)
##NLP can also be called seq2seq problems

###Checking for GPU


In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-c3f1d01c-3222-3d2f-8e7c-2a3a1c7c95a1)


In [2]:
##Get helper functions
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-07-23 20:20:03--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-07-23 20:20:03 (111 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [4]:
##Get a text dataset
###Using the Kaggle Intro to NLP dataset, samples of tweets labeled hazard or not
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2022-07-23 20:22:07--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.132.128, 74.125.202.128, 173.194.196.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.132.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-07-23 20:22:07 (143 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
unzip_data("nlp_getting_started.zip")

##Visualizing a text dataset

In [7]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
train_df["text"][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [9]:
train_df["text"][10]

'Three people died from the heat wave so far'

In [10]:
train_df["text"][100]

'.@NorwayMFA #Bahrain police had previously died in a road accident they were not killed by explosion https://t.co/gFJfgTodad'

In [12]:
#shuffle trianing df
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [13]:
#test df
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [14]:
#examples of each class
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [17]:
#total num of samples
len(train_df), len(test_df)

(7613, 3263)

In [19]:
#visualizing random samples
import random
random_index = random.randint(0, len(train_df)-5) #create random index
for row in train_df_shuffled[["text", "target"]][random_index:random_index +5 ].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}")

Target: 1 (real disaster
Text:
Only you can prevent forest fires. ???? http://t.co/rGYUaKc0dR
Target: 1 (real disaster
Text:
#NowPlaying at #orchardalley in #LES of #nyc 'bioterror- manufactured fear and state repression' @abcnorio #gardens http://t.co/Ba2rRXUgsG
Target: 1 (real disaster
Text:
Kakeru Teduka: Bfore 70years of today in Hiroshima it's exploded the one atomic bomd. It is so sad day.http://t.co/8Vzl1ns2iO
Target: 0 (not real disaster)
Text:
@CranBoonitz So going to make any bomb threats? @HereticOfEthics
Target: 0 (not real disaster)
Text:
Rite now man a tlk widout nuh curfew long side Aka cum fi steal di show itz a rubbery di whole a dem fi knw... Sound it *music*
